# Omission LFP Analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob
from collections import defaultdict
import re

In [3]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
import itertools

In [4]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity

## Inputs & Data

Explanation of each input and where it comes from.

In [5]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

In [6]:
BBOX_TO_ANCHOR=(1.5, 0.9)
LOC='upper right'

In [7]:
# variables for LFP extraction
FREQ_MIN=0.5
FREQ_MAX=300
FREQ=60
RESAMPLE_RATE=1000
TRIAL_DURATION=10

In [8]:
INPUT_VARIABLE = 1

TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 3
TIME_WINDOW_STEP = 1.5 

TRIAL_TIME_STAMP_DURATION = 1000*10

In [9]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5cm/s to 10cm/s"}

In [10]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 5cm/s", 1: "5cm/s to 10cm/s", 2: "10cm/s+"}

In [11]:
NUM_LINES = 3

In [12]:
# Generate colors from the "Blues" colormap
LOSING_COLORS = cm.Oranges(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
WINNING_COLORS = cm.Blues(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
REWARDED_COLORS = cm.Greens(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
OMISSION_COLORS = cm.Reds(np.linspace(0.5, 1, NUM_LINES))

In [13]:
BASELINE_OUTCOME_TO_COLOR = {'lose': "orange",
 'lose_baseline': LOSING_COLORS[0],
 'omission': "red",
 'omission_baseline': "hotpink",
 'rewarded': "green",
 'rewarded_baseline': REWARDED_COLORS[0],
 'win': "blue",
 'win_baseline': WINNING_COLORS[0]}

In [14]:
COMPETITIVE_OUTCOME_TO_COLOR = {'lose_comp': "orange", 
'lose_non_comp': "yellow",
'omission': "red",
'rewarded': "green",
'win_comp': "blue", 
'win_non_comp': WINNING_COLORS[0]}

In [15]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [16]:
CHANNEL_MAPPING_DF = pd.read_excel("../../channel_mapping.xlsx")
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

TONE_TIMESTAMP_DF = pd.read_excel("../../rce_tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


In [17]:
ALL_SESSION_DIR = list(set(['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_19/20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec'
                   ]))

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [18]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [19]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


### Getting the subject IDs from the file name

In [20]:
TONE_TIMESTAMP_DF = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")


In [21]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset="trial_outcome").reset_index(drop=True)

- Original timestamps are based on ephys recordings at 20kHz. The LFP will be at 1kHz, so we will need to divide all the timestamps by 20

In [22]:
all_trials_df["resampled_index"] = all_trials_df["time_stamp_index"] // 20

In [23]:
all_trials_df["recording_dir"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

- Getting a list of all the subjects through the recording name

In [24]:
all_trials_df["all_subjects"] = all_trials_df["recording_dir"].apply(lambda x: ["{}.{}".format(tup[0],tup[1]) for tup in re.findall(r'(\d+)-(\d+)', x.replace("_", "-"))[1:]])

In [25]:
all_trials_df["all_subjects"].head()

0    [1.1, 1.2]
1    [1.1, 1.2]
2    [1.1, 1.2]
3    [1.1, 1.2]
4    [1.1, 1.2]
Name: all_subjects, dtype: object

- Getting the current subject of the recording through the ending of the recording name file

In [26]:
all_trials_df["subject_info"].head()

0    1-1_t1b3L_box1
1    1-1_t1b3L_box1
2    1-1_t1b3L_box1
3    1-1_t1b3L_box1
4    1-1_t1b3L_box1
Name: subject_info, dtype: object

In [27]:
all_trials_df["current_subject"] = all_trials_df["subject_info"].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2]))

In [28]:
all_trials_df.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,trial_thorax_velocity,trial_chunked_thorax_velocity,baseline_chunked_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_baseline_thorax_velocity,binned_trial_thorax_velocity,resampled_index
0,4509412,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,1030879,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,lose_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[5.854587674773412, 5.491207535476736, 5.11749...","[4.204500001605147, 6.544647570039476, 1.57793...","[1.0482043233623093, 0.31708611003393056, 0.36...","[(1030879, 1050879), (1050879, 1070879), (1070...","[(830879, 850879), (850879, 870879), (870879, ...","[(51543, 52543), (52543, 53543), (53543, 54543...","[(41543, 42543), (42543, 43543), (43543, 44543...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]",51543
1,6909411,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,3430878,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[2.3816625552688544, 2.3175606442761967, 2.218...","[1.5520441045782087, 4.256821128528769, 11.997...","[0.24093936568383498, 0.1201192423141825, 0.23...","[(3430878, 3450878), (3450878, 3470878), (3470...","[(3230878, 3250878), (3250878, 3270878), (3270...","[(171543, 172543), (172543, 173543), (173543, ...","[(161543, 162543), (162543, 163543), (163543, ...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 0]","[0, 0, 2, 2, 0, 0, 0, 0, 0, 0]",171543
2,9209413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,5730880,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.5555477757700004, 0.4158707933887352, 0.265...","[1.2935699563401366, 1.8336016533619215, 7.857...","[5.100726428703984, 3.878556470230757, 2.49962...","[(5730880, 5750880), (5750880, 5770880), (5770...","[(5530880, 5550880), (5550880, 5570880), (5570...","[(286544, 287544), (287544, 288544), (288544, ...","[(276544, 277544), (277544, 278544), (278544, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",286544
3,11009410,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,7530877,20230617_115521_standard_comp_to_omission_D1_s...,7516,1.0,1-1_t1b3L_box1,lose_non_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.28520323663080543, 0.3655771428805912, 0.43...","[0.6892678457974358, 4.623066029743537, 4.8167...","[1.6265370368079863, 1.939419253786388, 0.7282...","[(7530877, 7550877), (7550877, 7570877), (7570...","[(7330877, 7350877), (7350877, 7370877), (7370...","[(376543, 377543), (377543, 378543), (378543, ...","[(366543, 367543), (367543, 368543), (368543, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 2, 2, 0, 0, 1, 2]",376543
4,12109413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,8630880,20230617_115521_standard_comp_to_omission_D1_s...,8615,1.0,1-1_t1b3L_box1,win_non_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.10093605479378118, 0.11690904896013582, 0.1...","[0.09127850278450059, 0.09571389765080775, 0.0...","[3.822843311916576, 2.0652198252227167, 2.6389...","[(8630880, 8650880), (8650880, 8670880), (8670...","[(8430880, 8450880), (8450880, 8470880), (8470...","[(431544, 432544), (432544, 433544), (433544, ...","[(421544, 422544), (422544, 423544), (423544, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",431544


In [29]:
all_trials_df.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'video_name', 'all_subjects',
       'current_subject', 'trial_outcome', 'lfp_index',
       'baseline_lfp_timestamp_range', 'trial_lfp_timestamp_range',
       'baseline_ephys_timestamp_range', 'trial_ephys_timestamp_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'trial_number',
       'Cohort', 'spike_interface_mPFC', 'spike_interface_vHPC',
       'spike_interface_BLA', 'spike_interface_LH', 'spike_interface_MD',
       'file_path', 'start_frame', 'stop_frame', 'individual_subj', 'all_subj',
       'recording_name', 'track_names', 'subject_id', 'corner_path',
       'corner_parts', 'bottom_width', 'top_width', 'right_height',
       'left_height', 'average_height', 'average_width', 'width_ratio',
       'height_ratio', 'rescaled_locations', 'box_top_left', 'box_bottom_left',
       'box_bottom

In [30]:
all_trials_df["trial_outcome"].unique()

array(['lose', 'win', 'rewarded', 'omission'], dtype=object)

- Labeling the trial as a winner or loser if the winner matches the subject id or not

In [31]:
all_trials_df.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,trial_thorax_velocity,trial_chunked_thorax_velocity,baseline_chunked_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_baseline_thorax_velocity,binned_trial_thorax_velocity,resampled_index
0,4509412,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,1030879,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,lose_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[5.854587674773412, 5.491207535476736, 5.11749...","[4.204500001605147, 6.544647570039476, 1.57793...","[1.0482043233623093, 0.31708611003393056, 0.36...","[(1030879, 1050879), (1050879, 1070879), (1070...","[(830879, 850879), (850879, 870879), (870879, ...","[(51543, 52543), (52543, 53543), (53543, 54543...","[(41543, 42543), (42543, 43543), (43543, 44543...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]",51543
1,6909411,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,3430878,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[2.3816625552688544, 2.3175606442761967, 2.218...","[1.5520441045782087, 4.256821128528769, 11.997...","[0.24093936568383498, 0.1201192423141825, 0.23...","[(3430878, 3450878), (3450878, 3470878), (3470...","[(3230878, 3250878), (3250878, 3270878), (3270...","[(171543, 172543), (172543, 173543), (173543, ...","[(161543, 162543), (162543, 163543), (163543, ...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 0]","[0, 0, 2, 2, 0, 0, 0, 0, 0, 0]",171543
2,9209413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,5730880,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.5555477757700004, 0.4158707933887352, 0.265...","[1.2935699563401366, 1.8336016533619215, 7.857...","[5.100726428703984, 3.878556470230757, 2.49962...","[(5730880, 5750880), (5750880, 5770880), (5770...","[(5530880, 5550880), (5550880, 5570880), (5570...","[(286544, 287544), (287544, 288544), (288544, ...","[(276544, 277544), (277544, 278544), (278544, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",286544
3,11009410,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,7530877,20230617_115521_standard_comp_to_omission_D1_s...,7516,1.0,1-1_t1b3L_box1,lose_non_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.28520323663080543, 0.3655771428805912, 0.43...","[0.6892678457974358, 4.623066029743537, 4.8167...","[1.6265370368079863, 1.939419253786388, 0.7282...","[(7530877, 7550877), (7550877, 7570877), (7570...","[(7330877, 7350877), (7350877, 7370877), (7370...","[(376543, 377543), (377543, 378543), (378543, ...","[(366543, 367543), (367543, 368543), (368543, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 2, 2, 0, 0, 1, 2]",376543
4,12109413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,8630880,20230617_115521_standard_comp_to_omission_D1_s...,8615,1.0,1-1_t1b3L_box1,win_non_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.10093605479378118, 0.11690904896013582, 0.1...","[0.09127850278450059, 0.09571389765080775, 0.0...","[3.822843311916576, 2.0652198252227167, 2.6389...","[(8630880, 8650880), (8650880, 8670880), (8670...","[(8430880, 8450880), (8450880, 8470880), (8470...","[(431544, 432544), (432544, 433544), (433544, ...","[(421544, 422544), (422544, 423544), (423544, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",431544


In [32]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in all_trials_df["competition_closeness"].unique()}

In [33]:
competition_closeness_map

{'lose_comp': 'comp',
 'win_comp': 'comp',
 'lose_non_comp': 'comp',
 'win_non_comp': 'comp',
 'rewarded': 'comp',
 'omission': 'comp'}

In [34]:
all_trials_df["competition_closeness"] = all_trials_df["competition_closeness"].map(competition_closeness_map)

In [35]:
all_trials_df["competition_closeness"]

0      comp
1      comp
2      comp
3      comp
4      comp
       ... 
235    comp
236    comp
237    comp
238    comp
239    comp
Name: competition_closeness, Length: 240, dtype: object

In [36]:
all_trials_df["competition_closeness"] = all_trials_df.apply(lambda x: "_".join([str(x["trial_outcome"]), str(x["competition_closeness"])]).strip("nan").strip("_"), axis=1)

### Extracting the LFP

In [37]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, "*.rec")):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id="ECU")
            current_recording = se.read_spikegadgets(recording_path, stream_id="trodes")
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=0.5, freq_max=300)
            current_recording = sp.notch_filter(current_recording, freq=60)
            current_recording = sp.resample(current_recording, resample_rate=1000)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except:
            pass



20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged
20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged


- Filtering for all trials that have labels

In [38]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [39]:
all_trials_df.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,trial_thorax_velocity,trial_chunked_thorax_velocity,baseline_chunked_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_baseline_thorax_velocity,binned_trial_thorax_velocity,resampled_index
0,4509412,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,1030879,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,lose_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[5.854587674773412, 5.491207535476736, 5.11749...","[4.204500001605147, 6.544647570039476, 1.57793...","[1.0482043233623093, 0.31708611003393056, 0.36...","[(1030879, 1050879), (1050879, 1070879), (1070...","[(830879, 850879), (850879, 870879), (870879, ...","[(51543, 52543), (52543, 53543), (53543, 54543...","[(41543, 42543), (42543, 43543), (43543, 44543...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]",51543
1,6909411,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,3430878,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[2.3816625552688544, 2.3175606442761967, 2.218...","[1.5520441045782087, 4.256821128528769, 11.997...","[0.24093936568383498, 0.1201192423141825, 0.23...","[(3430878, 3450878), (3450878, 3470878), (3470...","[(3230878, 3250878), (3250878, 3270878), (3270...","[(171543, 172543), (172543, 173543), (173543, ...","[(161543, 162543), (162543, 163543), (163543, ...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 0]","[0, 0, 2, 2, 0, 0, 0, 0, 0, 0]",171543
2,9209413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,5730880,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.5555477757700004, 0.4158707933887352, 0.265...","[1.2935699563401366, 1.8336016533619215, 7.857...","[5.100726428703984, 3.878556470230757, 2.49962...","[(5730880, 5750880), (5750880, 5770880), (5770...","[(5530880, 5550880), (5550880, 5570880), (5570...","[(286544, 287544), (287544, 288544), (288544, ...","[(276544, 277544), (277544, 278544), (278544, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",286544
3,11009410,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,7530877,20230617_115521_standard_comp_to_omission_D1_s...,7516,1.0,1-1_t1b3L_box1,lose_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.28520323663080543, 0.3655771428805912, 0.43...","[0.6892678457974358, 4.623066029743537, 4.8167...","[1.6265370368079863, 1.939419253786388, 0.7282...","[(7530877, 7550877), (7550877, 7570877), (7570...","[(7330877, 7350877), (7350877, 7370877), (7370...","[(376543, 377543), (377543, 378543), (378543, ...","[(366543, 367543), (367543, 368543), (368543, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 2, 2, 0, 0, 1, 2]",376543
4,12109413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,8630880,20230617_115521_standard_comp_to_omission_D1_s...,8615,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.10093605479378118, 0.11690904896013582, 0.1...","[0.09127850278450059, 0.09571389765080775, 0.0...","[3.822843311916576, 2.0652198252227167, 2.6389...","[(8630880, 8650880), (8650880, 8670880), (8670...","[(8430880, 8450880), (8450880, 8470880), (8470...","[(431544, 432544), (432544, 433544), (433544, ...","[(421544, 422544), (422544, 423544), (423544, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",431544


In [40]:
all_trials_df["trial_outcome"].unique()

array(['lose', 'win', 'rewarded', 'omission'], dtype=object)

In [42]:
recording_name_to_all_ch_lfp.keys()

dict_keys(['20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged', '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged', '20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged', '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged', '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged', '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged', '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged', '20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged'])

In [43]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


## Power Calculation

In [44]:
all_trials_df.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,trial_thorax_velocity,trial_chunked_thorax_velocity,baseline_chunked_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_baseline_thorax_velocity,binned_trial_thorax_velocity,resampled_index
0,4509412,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,1030879,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,lose_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[5.854587674773412, 5.491207535476736, 5.11749...","[4.204500001605147, 6.544647570039476, 1.57793...","[1.0482043233623093, 0.31708611003393056, 0.36...","[(1030879, 1050879), (1050879, 1070879), (1070...","[(830879, 850879), (850879, 870879), (870879, ...","[(51543, 52543), (52543, 53543), (53543, 54543...","[(41543, 42543), (42543, 43543), (43543, 44543...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]",51543
1,6909411,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,3430878,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[2.3816625552688544, 2.3175606442761967, 2.218...","[1.5520441045782087, 4.256821128528769, 11.997...","[0.24093936568383498, 0.1201192423141825, 0.23...","[(3430878, 3450878), (3450878, 3470878), (3470...","[(3230878, 3250878), (3250878, 3270878), (3270...","[(171543, 172543), (172543, 173543), (173543, ...","[(161543, 162543), (162543, 163543), (163543, ...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 0]","[0, 0, 2, 2, 0, 0, 0, 0, 0, 0]",171543
2,9209413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,5730880,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.5555477757700004, 0.4158707933887352, 0.265...","[1.2935699563401366, 1.8336016533619215, 7.857...","[5.100726428703984, 3.878556470230757, 2.49962...","[(5730880, 5750880), (5750880, 5770880), (5770...","[(5530880, 5550880), (5550880, 5570880), (5570...","[(286544, 287544), (287544, 288544), (288544, ...","[(276544, 277544), (277544, 278544), (278544, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",286544
3,11009410,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,7530877,20230617_115521_standard_comp_to_omission_D1_s...,7516,1.0,1-1_t1b3L_box1,lose_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.28520323663080543, 0.3655771428805912, 0.43...","[0.6892678457974358, 4.623066029743537, 4.8167...","[1.6265370368079863, 1.939419253786388, 0.7282...","[(7530877, 7550877), (7550877, 7570877), (7570...","[(7330877, 7350877), (7350877, 7370877), (7370...","[(376543, 377543), (377543, 378543), (378543, ...","[(366543, 367543), (367543, 368543), (368543, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 2, 2, 0, 0, 1, 2]",376543
4,12109413,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,8630880,20230617_115521_standard_comp_to_omission_D1_s...,8615,1.0,1-1_t1b3L_box1,win_comp,20230617_115521_standard_comp_to_omission_D1_s...,...,"[0.10093605479378118, 0.11690904896013582, 0.1...","[0.09127850278450059, 0.09571389765080775, 0.0...","[3.822843311916576, 2.0652198252227167, 2.6389...","[(8630880, 8650880), (8650880, 8670880), (8670...","[(8430880, 8450880), (8450880, 8470880), (8470...","[(431544, 432544), (432544, 433544), (433544, ...","[(421544, 422544), (422544, 423544), (423544, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",431544


In [50]:
all_trials_df.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'video_name', 'all_subjects',
       'current_subject', 'trial_outcome', 'lfp_index',
       'baseline_lfp_timestamp_range', 'trial_lfp_timestamp_range',
       'baseline_ephys_timestamp_range', 'trial_ephys_timestamp_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'trial_number',
       'Cohort', 'spike_interface_mPFC', 'spike_interface_vHPC',
       'spike_interface_BLA', 'spike_interface_LH', 'spike_interface_MD',
       'file_path', 'start_frame', 'stop_frame', 'individual_subj', 'all_subj',
       'recording_name', 'track_names', 'subject_id', 'corner_path',
       'corner_parts', 'bottom_width', 'top_width', 'right_height',
       'left_height', 'average_height', 'average_width', 'width_ratio',
       'height_ratio', 'rescaled_locations', 'box_top_left', 'box_bottom_left',
       'box_bottom

- Adding all the brain region to ch information

In [54]:
cols_to_use = all_trials_df.columns.difference(CHANNEL_MAPPING_DF.columns)


In [56]:
channel_map_and_all_trials_df = all_trials_df[cols_to_use].merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

- Linking up all LFP calculations with all the trials

In [57]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [58]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [59]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [60]:
channel_map_and_all_trials_df

,all_subj,all_subjects,average_height,average_width,baseline_chunked_ephys_timestamp,baseline_chunked_lfp_index,baseline_chunked_thorax_velocity,baseline_ephys_timestamp_range,baseline_lfp_timestamp_range,baseline_thorax_velocity,...,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD,all_ch_lfp
0,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(830879, 850879), (850879, 870879), (870879, ...","[(41543, 42543), (42543, 43543), (43543, 44543...","[1.0482043233623093, 0.31708611003393056, 0.36...","(830879, 1030879)","(41543, 51543)","[1.740210085775107, 1.7046357778943324, 1.6492...",...,16,17,18,19,5.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
1,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(3230878, 3250878), (3250878, 3270878), (3270...","[(161543, 162543), (162543, 163543), (163543, ...","[0.24093936568383498, 0.1201192423141825, 0.23...","(3230878, 3430878)","(161543, 171543)","[0.20814043433428542, 0.22314126629182324, 0.2...",...,16,17,18,19,5.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
2,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(5530880, 5550880), (5550880, 5570880), (5570...","[(276544, 277544), (277544, 278544), (278544, ...","[5.100726428703984, 3.878556470230757, 2.49962...","(5530880, 5730880)","(276544, 286544)","[5.587991473689541, 5.787334491104469, 5.97825...",...,16,17,18,19,5.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
3,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(7330877, 7350877), (7350877, 7370877), (7370...","[(366543, 367543), (367543, 368543), (368543, ...","[1.6265370368079863, 1.939419253786388, 0.7282...","(7330877, 7530877)","(366543, 376543)","[1.5079694415257687, 1.67290847806871, 1.80495...",...,16,17,18,19,5.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
4,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(8430880, 8450880), (8450880, 8470880), (8470...","[(421544, 422544), (422544, 423544), (423544, ...","[3.822843311916576, 2.0652198252227167, 2.6389...","(8430880, 8630880)","(421544, 431544)","[4.401859801710984, 4.594906485228284, 4.73967...",...,16,17,18,19,5.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1.2_1.4,"[1.4, 1.2]",263.090950,212.292023,"[(61888794, 61908794), (61908794, 61928794), (...","[(3094439, 3095439), (3095439, 3096439), (3096...","[0.591518062774352, 1.8651797757281146, 2.2240...","(61888794, 62088794)","(3094439, 3104439)","[0.15083632715470466, 0.2763562562227468, 0.43...",...,15,14,13,12,15.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
236,1.2_1.4,"[1.4, 1.2]",263.090950,212.292023,"[(63388812, 63408812), (63408812, 63428812), (...","[(3169440, 3170440), (3170440, 3171440), (3171...","[0.9576044353368744, 2.244484229703428, 1.9594...","(63388812, 63588812)","(3169440, 3179440)","[0.26954963853110736, 0.5336390342987469, 0.77...",...,15,14,13,12,15.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
237,1.2_1.4,"[1.4, 1.2]",263.090950,212.292023,"[(65388836, 65408836), (65408836, 65428836), (...","[(3269441, 3270441), (3270441, 3271441), (3271...","[1.4424292568325987, 2.460626790423604, 4.1534...","(65388836, 65588836)","(3269441, 3279441)","[0.6389643792864463, 0.8216919700151519, 1.130...",...,15,14,13,12,15.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....
238,1.2_1.4,"[1.4, 1.2]",263.090950,212.292023,"[(67288865, 67308865), (67308865, 67328865), (...","[(3364443, 3365443), (3365443, 3366443), (3366...","[0.9420699884829801, 2.822560619834402, 2.4673...","(67288865, 67488865)","(3364443, 3374443)","[0.8140859487059277, 0.7914479897581302, 0.984...",...,15,14,13,12,15.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1 segments - 1....


In [61]:
melted_channel_map_and_all_trials_df = channel_map_and_all_trials_df.melt(id_vars=id_cols ,value_vars=brain_region_col, var_name='brain_region', value_name='channel')


In [62]:
melted_channel_map_and_all_trials_df.head()

,all_subj,all_subjects,average_height,average_width,baseline_chunked_ephys_timestamp,baseline_chunked_lfp_index,baseline_chunked_thorax_velocity,baseline_ephys_timestamp_range,baseline_lfp_timestamp_range,baseline_thorax_velocity,...,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel
0,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(830879, 850879), (850879, 870879), (870879, ...","[(41543, 42543), (42543, 43543), (43543, 44543...","[1.0482043233623093, 0.31708611003393056, 0.36...","(830879, 1030879)","(41543, 51543)","[1.740210085775107, 1.7046357778943324, 1.6492...",...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
1,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(3230878, 3250878), (3250878, 3270878), (3270...","[(161543, 162543), (162543, 163543), (163543, ...","[0.24093936568383498, 0.1201192423141825, 0.23...","(3230878, 3430878)","(161543, 171543)","[0.20814043433428542, 0.22314126629182324, 0.2...",...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
2,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(5530880, 5550880), (5550880, 5570880), (5570...","[(276544, 277544), (277544, 278544), (278544, ...","[5.100726428703984, 3.878556470230757, 2.49962...","(5530880, 5730880)","(276544, 286544)","[5.587991473689541, 5.787334491104469, 5.97825...",...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
3,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(7330877, 7350877), (7350877, 7370877), (7370...","[(366543, 367543), (367543, 368543), (368543, ...","[1.6265370368079863, 1.939419253786388, 0.7282...","(7330877, 7530877)","(366543, 376543)","[1.5079694415257687, 1.67290847806871, 1.80495...",...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
4,1.1_1.2,"[1.1, 1.2]",264.959511,212.446672,"[(8430880, 8450880), (8450880, 8470880), (8470...","[(421544, 422544), (422544, 423544), (423544, ...","[3.822843311916576, 2.0652198252227167, 2.6389...","(8430880, 8630880)","(421544, 431544)","[4.401859801710984, 4.594906485228284, 4.73967...",...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0


In [63]:
melted_channel_map_and_all_trials_df.tail()

,all_subj,all_subjects,average_height,average_width,baseline_chunked_ephys_timestamp,baseline_chunked_lfp_index,baseline_chunked_thorax_velocity,baseline_ephys_timestamp_range,baseline_lfp_timestamp_range,baseline_thorax_velocity,...,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel
1195,1.2_1.4,"[1.4, 1.2]",263.09095,212.292023,"[(61888794, 61908794), (61908794, 61928794), (...","[(3094439, 3095439), (3095439, 3096439), (3096...","[0.591518062774352, 1.8651797757281146, 2.2240...","(61888794, 62088794)","(3094439, 3104439)","[0.15083632715470466, 0.2763562562227468, 0.43...",...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1196,1.2_1.4,"[1.4, 1.2]",263.09095,212.292023,"[(63388812, 63408812), (63408812, 63428812), (...","[(3169440, 3170440), (3170440, 3171440), (3171...","[0.9576044353368744, 2.244484229703428, 1.9594...","(63388812, 63588812)","(3169440, 3179440)","[0.26954963853110736, 0.5336390342987469, 0.77...",...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1197,1.2_1.4,"[1.4, 1.2]",263.09095,212.292023,"[(65388836, 65408836), (65408836, 65428836), (...","[(3269441, 3270441), (3270441, 3271441), (3271...","[1.4424292568325987, 2.460626790423604, 4.1534...","(65388836, 65588836)","(3269441, 3279441)","[0.6389643792864463, 0.8216919700151519, 1.130...",...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1198,1.2_1.4,"[1.4, 1.2]",263.09095,212.292023,"[(67288865, 67308865), (67308865, 67328865), (...","[(3364443, 3365443), (3365443, 3366443), (3366...","[0.9420699884829801, 2.822560619834402, 2.4673...","(67288865, 67488865)","(3364443, 3374443)","[0.8140859487059277, 0.7914479897581302, 0.984...",...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1199,1.2_1.4,"[1.4, 1.2]",263.09095,212.292023,"[(68288874, 68308874), (68308874, 68328874), (...","[(3414443, 3415443), (3415443, 3416443), (3416...","[2.1168894696324845, 1.2529255426378296, 0.547...","(68288874, 68488874)","(3414443, 3424443)","[1.9077957886068988, 1.8184625964386179, 1.703...",...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0


- Getting the traces for each trial for each brain region

In [64]:
melted_channel_map_and_all_trials_df["channel"] = melted_channel_map_and_all_trials_df["channel"].astype(int).astype(str)
melted_channel_map_and_all_trials_df["resampled_index"] = melted_channel_map_and_all_trials_df["resampled_index"].astype(int)

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"] = melted_channel_map_and_all_trials_df.apply(lambda x: 
[x["all_ch_lfp"].get_traces(channel_ids=[x["channel"]], start_frame=chunk_index[0], end_frame=chunk_index[1]).T[0] for chunk_index in x["chunked_lfp_index"]], axis=1)

In [ ]:
len(melted_channel_map_and_all_trials_df["chunked_current_trace"].iloc[0])

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"]

In [ ]:
melted_channel_map_and_all_trials_df["chunk_id"] = [list(range(0,10))] * len(melted_channel_map_and_all_trials_df)

In [ ]:
exploded_channel_map_and_all_trials_df = melted_channel_map_and_all_trials_df.explode(["current_chunked_thorax_velocity", "chunked_current_trace", "binned_current_thorax_velocity", "chunked_lfp_index",	"chunked_ephys_timestamp", "chunk_id"])

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: len(x))


In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: x[:RESAMPLE_RATE])

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"]

- Calcuating the power at each frequency band

In [ ]:
exploded_channel_map_and_all_trials_df["multitaper"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: Multitaper(time_series=x,                                                                                        sampling_frequency=RESAMPLE_RATE))

In [ ]:
exploded_channel_map_and_all_trials_df["connectivity"] = exploded_channel_map_and_all_trials_df["multitaper"].apply(lambda x: Connectivity.from_multitaper(x))

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.frequencies)

In [ ]:
exploded_channel_map_and_all_trials_df["power"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.power().squeeze())

In [ ]:
exploded_channel_map_and_all_trials_df.head()

In [ ]:
exploded_channel_map_and_all_trials_df.tail()

# Plotting by velocity

- Grouping all the rows(1s segment of trials) that share the same brain region, whether its before or after the tone, and velocity grouping

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_or_baseline', 'binned_current_thorax_velocity']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

- Calculating the mean and sem of the LFP power

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.legend(loc="lower left")
    
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    # plt.ylim(0, 0.005)
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


## Plotting all the lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["trial_outcome"].unique()

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="lower left")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="upper right")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="upper right")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"] = exploded_channel_map_and_all_trials_df["power"].apply(lambda x: x[4:13])

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"].iloc[0]

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"] = exploded_channel_map_and_all_trials_df["theta_power"].apply(lambda x: np.mean(x))

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"]

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["all_thorax_velocity"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df.info()

In [ ]:
exploded_channel_map_and_all_trials_df.drop(columns=["multitaper", "connectivity", 'all_thorax_locations', 'all_thorax_velocity', 'all_ch_lfp']).to_pickle("./proc/rce_1_sec_interval_lfp_power_spectra_and_velocity.pkl")

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"].iloc[0][4:13]

In [ ]:
grouped_all_trials_df["power"].iloc[0].shape

In [ ]:
grouped_all_trials_df["power"].iloc

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)   
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.008) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the competitiveness and lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'competition_closeness', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["competition_closeness"].unique()

# Drawing plots all together

In [ ]:
region_df

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)  
    plt.yscale("log")
    plt.ylim(0.01, 0.1) 


    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
            
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'competition_closeness', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.25) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
            
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["current_thorax_velocity"]

In [ ]:
melted_channel_map_and_all_trials_df["subject_id"].unique()

# Drawing Spectogram

In [ ]:
exploded_channel_map_and_all_trials_df["power"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df["brain_region"]

In [ ]:
exploded_channel_map_and_all_trials_df[""]

In [ ]:
grouped_all_trials_df

In [ ]:
raise ValueError()

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["recording_file"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["subject_id"].unique()